# -- DAY 6 --


In [1]:
# --- Part One ---

import io
from icecream import ic

test = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""

retvalue = 0

DIREZIONE = ["up", "right", "down", "left"]
MOVEMENT = { "up": (0, -1), "right": (1, 0), "down": (0, 1), "left": (-1, 0) }

class Map:
    """
        La mappa consiste di una griglia di caratteri, la quale viene importata dal file o da una stringa.
        Le coordinate x, y rappresentano rispettivamente una posizione orizzontale e verticale.
        Contiene degli ostacoli ("#") e una posizione di partenza ("^")
        La classe è anche rappresentata una guardia che si muove nella griglia. 
        La sua posizione è aggiornata ogni volta che il guardia si muove. Viene anche mantenuta la sua direzione.
        La posizione None indica non è presente nella griglia.
    """
    def __init__(self, grid):
        self.guard = {"pos": None, "dir": 0}
        self.visited = set()
        self.grid = self.import_grid(grid)
        self.map_size = len(self.grid[0]), len(self.grid)
        self.obstacles, self.guard["pos"] = self._map_grid(self.grid)
        if self.guard["pos"]:
            self.visited.add(self.guard["pos"])

        #print (self.guard)


    def import_grid(self,grid):
        if isinstance(grid, io.TextIOBase):
            #print ("hai passato un file")
            return [list(x.strip()) for x in grid]
        elif isinstance(grid, str):
            # print ("hai passato una stringa")
            return [list(x) for x in grid.splitlines()]
        elif isinstance(grid, list):
            # print ("hai passato una lista")
            return grid
        else:
            print ("grid must be a file, a string or a list")
    
    def _map_grid(self, grid):
        obstacles = {(x, y): grid[y][x] for y in range(len(grid)) for x in range(len(grid[0])) if grid[y][x] == "#"}
        position = [(x, y) for y in range(len(grid)) for x in range(len(grid[0])) if grid[y][x] == "^"]
        if len(position) != 1:
            raise Exception("there must be only one position")
        return obstacles, position[0]
    
    def guard_step(self) -> bool:
        """
            move the guard straight in the direction he is facing
        """
        pos = self.guard["pos"]
        move = MOVEMENT[DIREZIONE[self.guard["dir"]]]
        new_pos = pos[0] + move[0], pos[1] + move[1]
        if new_pos in self.obstacles:
            # DEBUG ic(new_pos,stp,self.guard["pos"])
            return False
        if new_pos[0] < 0 or new_pos[1] < 0 or new_pos[0] >= self.map_size[0] or new_pos[1] >= self.map_size[1]:
            # print("out of map")
            self.guard["pos"] = None
            return False
        self.guard["pos"] = new_pos
        return True

        # DEBUG print (self.guard)

    def guard_move(self, steps = None) -> int:
        """ 
            move the guard straight in the direction he is facing
            stops wheb he reaches an obstacle
            steps is the number of steps to move, None means infinite
            return number of steps
        """
        # DEBUG ic("gard move", self.guard["dir"])
        stp = 0 # steps done 
        while steps is None or stp < steps:
            if not self.guard_step():
                return stp
            self.visited.add(self.guard["pos"])
            stp += 1

        return stp

    def guard_rotate(self, dir = 1):
        """
            rotate right if dir = 1 (default), left if dir = -1
            return the new direction
        """
        if dir == 1 or dir == -1:
            self.guard["dir"] = (self.guard["dir"] + dir) % 4
        return self.guard["dir"]

with open("input.txt","r") as file:

    room = Map(file)

    retvalue = 1

    while room.guard["pos"]:
    # for i in range(11):
        steps = room.guard_move()
        retvalue = retvalue + steps
        room.guard_rotate(1)
 
    retvalue = len(room.visited)
    #     room.guard_rotate(1)


print (f"The solution of Puzzle 1 is: {retvalue}")


The solution of Puzzle 1 is: 4982


In [ ]:
# --- Part Two ---

# Use of object defined in part 1

class Guard(Map):
    """
        Inheritance of the class Map
        Now visited keep track also of the direction
    """

    def __init__(self, grid):
        super().__init__(grid)
        self.visited = set()
        self.initial_guard = self.guard.copy()
        self.obstacles_origin = self.obstacles.copy()
        self.visited.add((self.guard["pos"], self.guard["dir"]))
        #print (self.guard)

    def add_obstable(self,x, y , c = "O"):
        self.obstacles[(x, y)] = c

    def restart(self):
        self.guard = self.initial_guard.copy()
        self.obstacles = self.obstacles_origin.copy()
        self.visited = set()
        if self.guard["pos"]:
            self.visited.add(self.guard["pos"])

    def guard_move(self, steps = None) -> int:
        """ 
            move the guard straight in the direction he is facing
            stops wheb he reaches an obstacle
            steps is the number of steps to move, None means infinite
            return number of steps (-1 if a loop is intercempted)
        """
        # DEBUG ic("gard move", self.guard["dir"])
        stp = 0 # steps done 
        while steps is None or stp < steps:
            if not self.guard_step():
                return stp
            if (self.guard["pos"], self.guard["dir"]) in self.visited:
                # print ("Found Loop")
                return -1
            self.visited.add((self.guard["pos"], self.guard["dir"]))
            stp += 1

        return stp

        #print (self.guard

with open("input.txt","r") as file:

    room = Guard(file)

    print (room.visited)

    retvalue = 0

    for y in range(room.map_size[1]):
        for x in range(room.map_size[0]):
            if (x, y) == room.guard["pos"]:
                # print (f"{x,y} posizione della guardia")
                continue
            if (x, y) in room.obstacles:
                # print (f"({x,y}) posizione di un ostacolo")
                continue
            room.add_obstable(x, y)        
            while room.guard["pos"]:
            #for i in range(11):
                steps = room.guard_move()
                if steps == -1:
                    retvalue += 1
                    #print(f"LOOP per ostacolo in ({x,y})")
                    break
                room.guard_rotate(1)
            else:
                # print(f"no loop per ostacolo in ({x,y})")
                pass
            room.restart()
                

    # retvalue = len(room.visited)

print (f"The solution of Puzzle 2 is: {retvalue}")


{((89, 84), 0)}
The solution of Puzzle 2 is: 1663
